# Physics-Informed Neural Networks

In this workshop, you will implement physics-informed neural networks (PINNs) and apply it to two different problems: a forward and a inverse one.

For both of them, most of the code is already written - the goal is for you to take away the most important concepts. As shown in the lecture, the physics-informed component is to be encoded via the loss function, and that will be the emphasis of this workshop.

In [ ]:
from utils import *

%matplotlib inline
plt.rcParams.update({"figure.figsize": (6, 4), "font.size": 12})

# Sets up GPU for PyTorch if available
device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f"Using device: {device}")

In [ ]:
config = {
    'use_tqdm': True,
    'seed': 2,
}

torch.manual_seed(config['seed'])
np.random.seed(config['seed'])


### Forward Problem: Burger's Equation

PINNs were first proposed in [Physics Informed Deep Learning (Part I): Data-driven Solutions of Nonlinear Partial Differential Equations](https://arxiv.org/abs/1711.10561) and [Physics Informed Deep Learning (Part II): Data-driven Discovery of Nonlinear Partial Differential Equations](https://arxiv.org/abs/1711.10566).

These papers suggest using neural networks to model phenomena in physics and give an example with a PINN for the Burger's equation, since this equation is simple to understand, but can be tricky to solve numerically. In the first part of this workshop we are going to solve a forward problem (data-driven solution) with a PINN for the Burger's equation.

A **forward problem** in the context of differential equations is simply this:

> **“Given a governing equation together with its initial and boundary conditions, predict the solution field.”**

In other words, we already know

1. **The physics**—the partial differential equation (PDE) itself (here, the viscous Burgers’ equation),
2. **The parameters**—for Burgers’ equation, the viscosity $\lambda$ is known,
3. **The constraints**—initial data $u(x,0)$ and boundary values on $x=-1$ and $x=1$.

#### Forward vs. Inverse

It's important not to conflate the **forward problem** (solve for $u$ when $\lambda$ and the PDE form are known) with the **inverse problem** (infer unknown parameters—like $\lambda$ or even the form of the nonlinearity—given measurements of $u$). PINNs tackle both, but in this first part we focus strictly on the **forward problem**, i.e., *data-driven solution* rather than *data-driven discovery*.

#### Your Task

Your task is to train a PINN $u_\theta(x,t)$ to approximate the solution on $(x,t)\in[-1,1]\times[0,1]$ by minimizing two losses over:

1. **IC/BC points** $\Phi_{\rm icbc} = \{(x_i,t_i)\}$:
   Enforce

   $$
     u_\theta(x_i,t_i) = u_{\rm icbc}(x_i,t_i),
     \quad
     u_{\rm icbc}(x,0) = -\sin(\pi x),\;
     u_{\rm icbc}(\pm1,t)=0.
   $$

2. **Collocation points** $\Phi_f = \{(x_j,t_j)\}$:
   Enforce the PDE residual

   $$
     r(x_j,t_j)
     = u_t + u\,u_x - \lambda\,u_{xx}
     = 0
   $$

In short, find $\theta$ such that

$$
\mathcal L 
= \underbrace{\frac1{|\Phi_{\rm icbc}|}\sum_{\Phi_{\rm icbc}}
\bigl(u_\theta - u_{\rm icbc}\bigr)^2}_{\text{IC/BC loss}}
+ \underbrace{\frac1{|\Phi_f|}\sum_{\Phi_f}r^2}_{\text{PDE residual loss}}
$$

is minimized.

In [ ]:
path = '../data/burgers_shock.mat'  # adjust path as needed
icbc_points = 100
f_points = 10000

x, t, X_mesh, T_mesh, u_target_mesh, Phi_icbc, u_icbc, Phi_f = load_data_burgers(
    path, icbc_points, f_points
)

# move data to GPU if available
Phi_icbc, u_icbc, Phi_f = [v.to(device) for v in (Phi_icbc, u_icbc, Phi_f)]
Phi_f.requires_grad_(True) # this is key for the loss function!

tensor([[ 0.6367,  0.6217],
        [-0.2016,  0.5471],
        [ 0.0759,  0.8639],
        ...,
        [ 0.0782,  0.2125],
        [-0.0916,  0.5380],
        [ 0.7190,  0.5947]], device='mps:0', requires_grad=True)

In [ ]:
class BurgersNet(nn.Module):
    def __init__(self, layers=(2, *[20]*8, 1)):
        super().__init__()
        modules = []
        for i in range(len(layers)-1):
            modules.append(nn.Linear(layers[i], layers[i+1]))
            if i < len(layers)-2:
                modules.append(nn.Tanh())
        self.net = nn.Sequential(*modules)
    def forward(self, x):
        return self.net(x)

model = BurgersNet().to(device)


Please note the loss terms:

1. **Data loss** on initial and boundary conditions (IC/BC):  
   $$
   \mathrm{MSE}_{\mathrm{IC/BC}} = \frac{1}{N_{\mathrm{ic}}}\sum_{i=1}^{N_{\mathrm{ic}}} \bigl(u_{\mathrm{pred}}(t_i,x_i) - u_{\mathrm{true}}(t_i,x_i)\bigr)^2
   $$

2. **PDE residual loss** at collocation points:  
   $$
   \mathrm{MSE}_{\mathrm{PDE}} = \frac{1}{N_{\mathrm{col}}}\sum_{j=1}^{N_{\mathrm{col}}}
   \bigl(u_t + u\,u_x - \nu\,u_{xx}\bigr)^2
   $$

The total loss is
$$
\mathcal{L} = \mathrm{MSE}_{\mathrm{IC/BC}} + \mathrm{MSE}_{\mathrm{PDE}}.
$$


In [ ]:
def pinn_loss(u_icbc_pred, u_icbc_true, u_f_pred, grads, viscosity=0.01/np.pi):
    # TODO: implement the loss function
    # you should return the total loss, but also the individual components
    ut, ux, uxx = grads
    mse_icbc = torch.mean((u_icbc_pred - u_icbc_true)**2)
    mse_pde = torch.mean((ut + u_f_pred * ux - viscosity * uxx)**2)
    total_loss = mse_icbc + mse_pde
    return total_loss, mse_icbc.item(), mse_pde.item()

In [ ]:
epochs = 2000
lr = 2e-3
scheduler_step = 2000
scheduler_gamma = 0.5
lbfgs_start = 1700

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)
loss_hist = []
metrics = {}

def closure():
    optimizer.zero_grad()

    # TODO: implement the autograd calls
    # The model calculates the output for the initial/boundary conditions
    # and also for the collocation points
    # You should also calculate the gradients of the collocation points
    # to use them in the loss function

    # IC/BC prediction
    u_icbc_pred = model(Phi_icbc)
    # Collocation prediction
    u_f_pred = model(Phi_f)
    gradients = grad(u_f_pred.sum(), Phi_f, create_graph=True)[0]
    ux = gradients[:, 0:1]
    ut = gradients[:, 1:2]
    uxx = grad(ux.sum(), Phi_f, create_graph=True)[0][:, 1:2]
    loss_val, l_icbc, l_pde = pinn_loss(u_icbc_pred, u_icbc, u_f_pred, (ut, ux, uxx))
    metrics['loss_val'] = loss_val
    metrics['l_icbc'] = l_icbc
    metrics['l_pde'] = l_pde
    loss_val.backward()
    return loss_val

In [ ]:
iterable = tqdm(range(1, epochs+1), disable=not config['use_tqdm'])
for epoch in iterable:
    if epoch == lbfgs_start:
        print("Switching to LBFGS optimizer")
        optimizer = torch.optim.LBFGS(
            model.parameters(),
            tolerance_grad=1e-6,
        )
    if epoch < lbfgs_start:
        loss = closure()
        optimizer.step()
        scheduler.step()
    else:
        loss = optimizer.step(closure)

    loss_hist.append((metrics['loss_val'].item(), metrics['l_icbc'], metrics['l_pde']))
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d} | Total: {loss_hist[-1][0]:.3e} "
              f"| ICBC: {loss_hist[-1][1]:.3e} | PDE: {loss_hist[-1][2]:.3e}")

  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch  100 | Total: 1.477e-01 | ICBC: 1.345e-01 | PDE: 1.325e-02
Epoch  200 | Total: 9.244e-02 | ICBC: 6.530e-02 | PDE: 2.715e-02
Epoch  300 | Total: 7.050e-02 | ICBC: 5.075e-02 | PDE: 1.974e-02
Epoch  400 | Total: 6.114e-02 | ICBC: 4.450e-02 | PDE: 1.663e-02
Epoch  500 | Total: 5.683e-02 | ICBC: 4.139e-02 | PDE: 1.543e-02
Epoch  600 | Total: 5.364e-02 | ICBC: 3.978e-02 | PDE: 1.386e-02
Epoch  700 | Total: 5.384e-02 | ICBC: 3.817e-02 | PDE: 1.567e-02
Epoch  800 | Total: 5.010e-02 | ICBC: 3.826e-02 | PDE: 1.184e-02
Epoch  900 | Total: 4.855e-02 | ICBC: 3.748e-02 | PDE: 1.107e-02
Epoch 1000 | Total: 4.764e-02 | ICBC: 3.707e-02 | PDE: 1.057e-02
Epoch 1100 | Total: 4.832e-02 | ICBC: 3.679e-02 | PDE: 1.153e-02
Epoch 1200 | Total: 4.738e-02 | ICBC: 3.791e-02 | PDE: 9.470e-03
Epoch 1300 | Total: 4.825e-02 | ICBC: 3.794e-02 | PDE: 1.031e-02
Epoch 1400 | Total: 6.996e-02 | ICBC: 5.434e-02 | PDE: 1.561e-02
Epoch 1500 | Total: 5.769e-02 | ICBC: 4.331e-02 | PDE: 1.438e-02
Epoch 1600 | Total: 5.431

In [ ]:
loss_arr = np.array(loss_hist)
plt.figure()
plt.semilogy(loss_arr[:,0], label='Total')
plt.semilogy(loss_arr[:,1], '--', label='ICBC')
plt.semilogy(loss_arr[:,2], '--', label='PDE')
plt.axvline(x=lbfgs_start, color='k', linestyle='--', label='LBFGS Start')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.title('Training Loss')
plt.show()

In [ ]:
with torch.no_grad():
    Phi_all = torch.from_numpy(
        np.hstack([X_mesh.reshape(-1,1), T_mesh.reshape(-1,1)])
    ).float().to(device)
    u_pred = model(Phi_all).cpu().numpy().reshape(u_target_mesh.shape)

fig, axes = plt.subplots(1, 2, figsize=(10,4))
# True solution
cf0 = axes[0].contourf(T_mesh, X_mesh, u_target_mesh, cmap='viridis')
axes[0].set_title('True Solution')
axes[0].set_xlabel('t')
axes[0].set_ylabel('x')
fig.colorbar(cf0, ax=axes[0])
# PINN prediction
cf1 = axes[1].contourf(T_mesh, X_mesh, u_pred, cmap='viridis')
axes[1].set_title('PINN Prediction')
axes[1].set_xlabel('t')
axes[1].set_ylabel('x')
fig.colorbar(cf1, ax=axes[1])

plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
line_true, = ax.plot([], [], 'b-', label='True')
line_pred, = ax.plot([], [], 'r--', label='NN pred')
ax.set_xlim(float(x.min()), float(x.max()))
ymin = min(u_target_mesh.min(), u_pred.min())
ymax = max(u_target_mesh.max(), u_pred.max())
ax.set_ylim(ymin, ymax)
ax.set_xlabel('x')
ax.set_ylabel('u')
ax.legend()

def init():
    line_true.set_data([], [])
    line_pred.set_data([], [])
    return line_true, line_pred

def update(frame):
    xx = x.flatten()
    y_true = u_target_mesh[frame, :]
    y_pred = u_pred[frame, :]
    line_true.set_data(xx, y_true)
    line_pred.set_data(xx, y_pred)
    ax.set_title(f't = {float(t[frame,0]):.3f}')
    return line_true, line_pred

ani = FuncAnimation(fig, update, frames=u_pred.shape[0],
                    init_func=init, blit=True, interval=100)

HTML(ani.to_jshtml())

In [ ]:
error_l2 = np.linalg.norm((u_pred - u_target_mesh).ravel())/np.linalg.norm(u_target_mesh.ravel())
print(f"Relative L2 Error: {error_l2:.3e}")